In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import uuid
from datetime import datetime
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicializar ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Variable para marcar si ya hemos aceptado cookies
cookies_aceptadas = False
# Función para aceptar cookies usando el ID del botón solo una vez
def aceptar_cookies():
    global cookies_aceptadas
    if not cookies_aceptadas:
        try:
            # Esperar hasta que el botón de aceptar cookies esté visible
            aceptar_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button"))
            )
            aceptar_button.click()
            print("Cookies aceptadas correctamente.")
            cookies_aceptadas = True
        except Exception as e:
            print(f"No se pudo aceptar las cookies: {e}")

# Función auxiliar para extraer texto o devolver 'N/A'
def obtener_texto_o_NA(selector_func, *args):
    try:
        elemento = selector_func(*args)
        return elemento.text if elemento else "N/A"
    except Exception:
        return "N/A"

# Función para obtener el precio
def obtener_precio():
    try:
        # Buscar el elemento que contiene el precio
        precio_elemento = driver.find_element(By.CSS_SELECTOR, ".price__value.jsPriceValue")
        # Extraer el texto del elemento y eliminar el contenido del span adicional
        precio_texto = precio_elemento.text.split()[0] + " €"  # Tomar el valor antes del <span>
        return precio_texto
    except Exception as e:
        print(f"Error al extraer precio: {e}")
        
# Función para obtener el número de fotos
def obtener_numero_fotos():
    try:
        # Buscar el elemento que contiene el número de fotos
        fotos_elemento = driver.find_element(By.CSS_SELECTOR, ".media-types-menu__number")
        fotos_numero = fotos_elemento.get_attribute("innerText")  # Obtener el texto entre las etiquetas <span>
        return fotos_numero
    except Exception as e:
        print(f"Error al extraer número de fotos: {e}")
        return "N/A"
        
# Lista de provincias tal como aparecen en pisos.com
provincias = [
    "a_coruna", "alava_araba", "albacete", "alicante", "almeria", "asturias", "avila", "badajoz",
    "barcelona", "burgos", "caceres", "cadiz", "cantabria", "castellon_castello", "ceuta", "ciudad_real", 
    "cordoba", "cuenca", "girona", "granada", "guadalajara", "guipuzcoa_gipuzkoa", "huelva", "huesca", "islas_baleares_illes_balears", "jaen", 
    "la_rioja", "las_palmas", "leon", "lleida", "lugo", "madrid", "malaga", "melilla", "murcia", "navarra_nafarroa",
    "ourense", "pais_vasco_frances_iparralde", "palencia", "pontevedra", "salamanca", "santa_cruz_de_tenerife","segovia", "sevilla", "soria", "tarragona",  
    "teruel", "toledo", "valencia", "valladolid", "vizcaya_bizkaia", "zamora", "zaragoza"
]

# Definir los nombres de los archivos CSV
csv_file_alquiler = "pisos_alquiler_data.csv"

# Crear y abrir el archivo CSV para alquiler
with open(csv_file_alquiler, mode='w', newline='', encoding='utf-8') as file_alquiler:

    writer_alquiler = csv.writer(file_alquiler)

    # Escribir las cabeceras de los archivos CSV (incluyen ID, Timestamp y las nuevas columnas)
    writer_alquiler.writerow([
        "ID", "Timestamp", "Provincia", "Título", "Precio", "Latitud", "Longitud", 
        "Superficie Construida", "Superficie Útil", "Habitaciones", "Baños", 
        "Planta", "Certificado Energético", "Número de Fotos", "Enlace"
        ])

    # Función para extraer publicaciones de alquiler por provincia
    def scrape_alquiler_provincia():
        for provincia in provincias:
            page_number = 1
            while True:
                try:
                    # Construir la URL de cada página de resultados por provincia
                    if page_number == 1:
                        # URL sin número de página para la primera página
                        url = f"https://www.pisos.com/alquiler/pisos-{provincia}/"
                    else:
                        # URL con número de página para la segunda página en adelante
                        url = f"https://www.pisos.com/alquiler/pisos-{provincia}/{page_number}/"
                    
                    driver.get(url)
                    time.sleep(5)  # Esperar a que la página cargue

                    # Aceptar cookies (solo la primera vez)
                    aceptar_cookies()

                    # Extraer los anuncios de propiedades
                    propiedades = driver.find_elements(By.CSS_SELECTOR, ".ad-preview")
                    if not propiedades:
                        break  # Si no hay propiedades, hemos llegado al final

                    for propiedad in propiedades:
                        try:
                            # Extraer el enlace de la propiedad
                            enlace = propiedad.find_element(By.CSS_SELECTOR, ".ad-preview__title").get_attribute("href")

                            # Abrir el enlace en una nueva pestaña
                            driver.execute_script(f"window.open('{enlace}', '_blank');")
                            driver.switch_to.window(driver.window_handles[-1])  # Cambiar a la última pestaña
                            try:
                                # Extraer datos con validación
                                titulo = obtener_texto_o_NA(driver.find_element, By.TAG_NAME, "h1")
                                precio = obtener_precio()  # Usar la nueva función de precio
                                latitud_longitud = driver.find_element(By.CSS_SELECTOR, "img[alt='mapa']").get_attribute("src").split("@")
                                latitud = latitud_longitud[0].split("_")[-1]
                                longitud = latitud_longitud[1].split("_")[0]
                                superficie_construida = obtener_texto_o_NA(driver.find_element, By.XPATH, "//span[contains(text(), 'Superficie construida')]/following-sibling::span")
                                superficie_util = obtener_texto_o_NA(driver.find_element, By.XPATH, "//span[contains(text(), 'Superficie útil')]/following-sibling::span")
                                habitaciones = obtener_texto_o_NA(driver.find_element, By.XPATH, "//span[contains(text(), 'Habitaciones')]/following-sibling::span")
                                banos = obtener_texto_o_NA(driver.find_element, By.XPATH, "//span[contains(text(), 'Baños')]/following-sibling::span")
                                planta = obtener_texto_o_NA(driver.find_element, By.XPATH, "//span[contains(text(), 'Planta')]/following-sibling::span")
                                certificado_energetico = obtener_texto_o_NA(driver.find_element, By.CSS_SELECTOR, ".energy-certificate p")
                                numero_fotos = obtener_numero_fotos()  # Usar la nueva función de número de fotos

                                # Generar un ID y timestamp únicos
                                unique_id = str(uuid.uuid4())
                                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                                # Guardar la información en el CSV de alquiler
                                writer_alquiler.writerow([
                                    unique_id, timestamp, provincia, titulo, precio, latitud, longitud, 
                                    superficie_construida, superficie_util, habitaciones, banos, 
                                    planta, certificado_energetico, numero_fotos, enlace
                                ])

                            finally:
                                # Asegurarse de cerrar la pestaña de la propiedad y regresar a la pestaña del listado
                                if len(driver.window_handles) > 1:
                                    driver.close()
                                time.sleep(2)  # Esperar un poco después de cerrar la pestaña
                                driver.switch_to.window(driver.window_handles[0])  # Cambiar de nuevo a la pestaña original

                        except Exception as e:
                            print(f"Error al extraer propiedad en {provincia} (alquiler): {e}")

                    # Ir a la siguiente página
                    page_number += 1
                    time.sleep(3)

                except Exception as e:
                    print(f"Error al navegar en {provincia} (alquiler): {e}")
                    break

    # Ejecutar la función para extraer los datos de alquiler
    scrape_alquiler_provincia()

# Cerrar el navegador después de finalizar
driver.quit()

print(f"Scraping finalizado. Los datos se han guardado en {csv_file_alquiler} (alquiler).")


Cookies aceptadas correctamente.
Error al extraer número de fotos: Message: no such element: Unable to locate element: {"method":"css selector","selector":".media-types-menu__number"}
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00625523+24195]
	(No symbol) [0x005BAA04]
	(No symbol) [0x004B2093]
	(No symbol) [0x004F6ED2]
	(No symbol) [0x004F711B]
	(No symbol) [0x005376F2]
	(No symbol) [0x0051AB84]
	(No symbol) [0x00535280]
	(No symbol) [0x0051A8D6]
	(No symbol) [0x004EBA27]
	(No symbol) [0x004EC43D]
	GetHandleVerifier [0x008ECE13+2938739]
	GetHandleVerifier [0x0093EC69+3274185]
	GetHandleVerifier [0x006B09C2+594722]
	GetHandleVerifier [0x006B7EDC+624700]
	(No symbol) [0x005C37CD]
	(No symbol) [0x005C0528]
	(No symbol) [0x005C06C5]
	(No symbol) [0x005B2CA6]
	BaseThreadInitThunk [0x77B8FCC9+25]
	RtlGetAppContainerNa

In [11]:
# Ruta del archivo CSV
archivo = 'pisos_alquiler_data.csv'

# Cargar el CSV usando el delimitador correcto
df = pd.read_csv(archivo, delimiter=';')

# Mostrar las primeras filas para verificar la carga correcta
print(df.head())


                                     ID         Timestamp Provincia  \
0  2ebc2089-fb32-4dcf-bc1f-48f952f9c7ab  18/10/2024 13:44  a_coruna   
1  4677a3e9-e1f0-4d6f-a905-e547184a7b80  18/10/2024 13:44  a_coruna   
2  b26707fe-a110-4b78-9bab-ae24afea9a02  18/10/2024 13:44  a_coruna   
3  4aa7369a-883f-46f4-9118-da91137cabec  18/10/2024 13:44  a_coruna   
4  08526e4e-b6ac-4861-9382-f59e73b23595  18/10/2024 13:44  a_coruna   

                                              Título   Precio  \
0                       Piso en alquiler en Ensanche  1.600 €   
1  Piso en alquiler en Avenida de Primo de Rivera, 8  1.550 €   
2  Piso en alquiler en Avenida de Pedro Barrié de...    980 €   
3  Piso en alquiler en Calle del Comandante Fontanes    950 €   
4  Apartamento en alquiler en Calle de Nuestra Se...    800 €   

                 Latitud                Longitud Superficie Construida  \
0            433.650.629             -84.068.429                168 m²   
1                433.596          

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15773 entries, 0 to 15772
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      15773 non-null  object 
 1   Timestamp               15773 non-null  object 
 2   Provincia               15773 non-null  object 
 3   Título                  15773 non-null  object 
 4   Precio                  15772 non-null  object 
 5   Latitud                 15772 non-null  object 
 6   Longitud                15772 non-null  object 
 7   Superficie Construida   15449 non-null  object 
 8   Superficie Útil         9247 non-null   object 
 9   Habitaciones            15456 non-null  float64
 10  Baños                   15530 non-null  float64
 11  Planta                  10722 non-null  object 
 12  Certificado Energético  15772 non-null  object 
 13  Número de Fotos         15612 non-null  float64
 14  Enlace                  15772 non-null

In [15]:
def sustituir_nulos(df):
    # Reemplazar todos los valores nulos por 'sin especificar'
    df.fillna('sin especificar', inplace=True)
    print("Todos los valores nulos han sido reemplazados por 'sin especificar'.")
    
    return df

# Aplicar la función al DataFrame
df = sustituir_nulos(df)

Todos los valores nulos han sido reemplazados por 'sin especificar'.


C:\Users\PC\AppData\Local\Temp\ipykernel_10028\1108492653.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sin especificar' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('sin especificar', inplace=True)


In [19]:
def limpiar_certificado_energetico(df):
    # Eliminar la cadena 'Clasificación: ' de la columna 'Certificado Energético'
    df['Certificado Energético'] = df['Certificado Energético'].str.replace('Clasificación: ', '', regex=False)
    
    return df

# Aplicar la función al DataFrame
df = limpiar_certificado_energetico(df)
df.head()

,ID,Timestamp,Provincia,Título,Precio,Latitud,Longitud,Superficie Construida,Superficie Útil,Habitaciones,Baños,Planta,Certificado Energético,Número de Fotos,Enlace
0,2ebc2089-fb32-4dcf-bc1f-48f952f9c7ab,18/10/2024 13:44,a_coruna,Piso en alquiler en Ensanche,1.600 €,433.650.629,-84.068.429,168 m²,140 m²,4.0,3.0,sin especificar,En trámite,17.0,https://www.pisos.com/alquilar/piso-ensanche15...
1,4677a3e9-e1f0-4d6f-a905-e547184a7b80,18/10/2024 13:44,a_coruna,"Piso en alquiler en Avenida de Primo de Rivera, 8",1.550 €,433.596,-84.036,75 m²,sin especificar,2.0,2.0,5ª,En trámite,20.0,https://www.pisos.com/alquilar/piso-catro_cami...
2,b26707fe-a110-4b78-9bab-ae24afea9a02,18/10/2024 13:44,a_coruna,Piso en alquiler en Avenida de Pedro Barrié de...,980 €,43.371.054.913,-8.404.348.532,100 m²,90 m²,3.0,2.0,2ª,Disponible,26.0,https://www.pisos.com/alquilar/piso-cidade_vel...
3,4aa7369a-883f-46f4-9118-da91137cabec,18/10/2024 13:44,a_coruna,Piso en alquiler en Calle del Comandante Fontanes,950 €,433.696.939,-84.053.941,95 m²,90 m²,2.0,2.0,1ª,Disponible,36.0,https://www.pisos.com/alquilar/piso-cidade_vel...
4,08526e4e-b6ac-4861-9382-f59e73b23595,18/10/2024 13:44,a_coruna,Apartamento en alquiler en Calle de Nuestra Se...,800 €,4.335.952.640.926.880,-8.414.446.644.177.610,60 m²,56 m²,1.0,1.0,1ª,Disponible,20.0,https://www.pisos.com/alquilar/apartamento-sag...


In [21]:
def limpiar_provincia(df):
    # Reemplazar guiones bajos por espacios en blanco
    df['Provincia'] = df['Provincia'].str.replace('_', ' ')
    
    # Reemplazar 'coruna' por 'coruña'
    df['Provincia'] = df['Provincia'].str.replace('coruna', 'coruña', regex=False)
    
    
    return df

# Aplicar la función al DataFrame
df = limpiar_provincia(df)
df.head()

,ID,Timestamp,Provincia,Título,Precio,Latitud,Longitud,Superficie Construida,Superficie Útil,Habitaciones,Baños,Planta,Certificado Energético,Número de Fotos,Enlace
0,2ebc2089-fb32-4dcf-bc1f-48f952f9c7ab,18/10/2024 13:44,a coruña,Piso en alquiler en Ensanche,1.600 €,433.650.629,-84.068.429,168 m²,140 m²,4.0,3.0,sin especificar,En trámite,17.0,https://www.pisos.com/alquilar/piso-ensanche15...
1,4677a3e9-e1f0-4d6f-a905-e547184a7b80,18/10/2024 13:44,a coruña,"Piso en alquiler en Avenida de Primo de Rivera, 8",1.550 €,433.596,-84.036,75 m²,sin especificar,2.0,2.0,5ª,En trámite,20.0,https://www.pisos.com/alquilar/piso-catro_cami...
2,b26707fe-a110-4b78-9bab-ae24afea9a02,18/10/2024 13:44,a coruña,Piso en alquiler en Avenida de Pedro Barrié de...,980 €,43.371.054.913,-8.404.348.532,100 m²,90 m²,3.0,2.0,2ª,Disponible,26.0,https://www.pisos.com/alquilar/piso-cidade_vel...
3,4aa7369a-883f-46f4-9118-da91137cabec,18/10/2024 13:44,a coruña,Piso en alquiler en Calle del Comandante Fontanes,950 €,433.696.939,-84.053.941,95 m²,90 m²,2.0,2.0,1ª,Disponible,36.0,https://www.pisos.com/alquilar/piso-cidade_vel...
4,08526e4e-b6ac-4861-9382-f59e73b23595,18/10/2024 13:44,a coruña,Apartamento en alquiler en Calle de Nuestra Se...,800 €,4.335.952.640.926.880,-8.414.446.644.177.610,60 m²,56 m²,1.0,1.0,1ª,Disponible,20.0,https://www.pisos.com/alquilar/apartamento-sag...


In [23]:
# Guardar el DataFrame en un archivo CSV
df.to_csv('pisos_alquiler_limpio.csv', index=False)


In [19]:
# Ruta del archivo CSV
archivo_ = 'pisos_venta.csv'

# Cargar el CSV usando el delimitador correcto
df_ = pd.read_csv(archivo_, delimiter=';')

# Mostrar las primeras filas para verificar la carga correcta
print(df_.head())


                                     ID         Timestamp Provincia  \
0  6f85c5bd-5fc6-452e-bf96-f300d94f74ba  19/10/2024 17:21  a_coruna   
1  ef1933dd-0707-4bd8-bfe8-0d2f00e90cbe  19/10/2024 17:21  a_coruna   
2  a47d9fc7-dd06-4d10-aa01-9022a0636e3a  19/10/2024 17:22  a_coruna   
3  f6a4795b-36f8-4beb-85c1-3b422df8736d  19/10/2024 17:22  a_coruna   
4  6fb83620-c719-47c9-a914-1b620206e41d  19/10/2024 17:22  a_coruna   

                                              Título     Precio  \
0        Chalet adosado en venta en Rúa Paulo Coelho  338.800 €   
1               Piso en venta en Calle Manuel Beiras  368.000 €   
2                Piso en venta en Calle de la Laguna  165.000 €   
3            Casa en venta en Lugar de Alborelle, 15  275.000 €   
4  Ático en venta en Calle del Alcalde Domingo Si...  146.700 €   

          Latitud        Longitud Superficie Construida Superficie Útil  \
0     428.906.606     -84.987.815                193 m²          161 m²   
1     428.697.438   

In [21]:
def sustituir_nulos(df_):
    # Reemplazar todos los valores nulos por 'sin especificar'
    df_.fillna('sin especificar', inplace=True)
    print("Todos los valores nulos han sido reemplazados por 'sin especificar'.")
    
    return df_

# Aplicar la función al DataFrame
df_ = sustituir_nulos(df_)

Todos los valores nulos han sido reemplazados por 'sin especificar'.


C:\Users\PC\AppData\Local\Temp\ipykernel_6572\806563267.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sin especificar' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_.fillna('sin especificar', inplace=True)


In [23]:
def limpiar_certificado_energetico(df_):
    # Eliminar la cadena 'Clasificación: ' de la columna 'Certificado Energético'
    df_['Certificado Energético'] = df_['Certificado Energético'].str.replace('Clasificación: ', '', regex=False)
    
    return df_

# Aplicar la función al DataFrame
df_ = limpiar_certificado_energetico(df_)
df_.head()

,ID,Timestamp,Provincia,Título,Precio,Latitud,Longitud,Superficie Construida,Superficie Útil,Habitaciones,Baños,Planta,Certificado Energético,Número de Fotos,Enlace,venta/alquiler
0,6f85c5bd-5fc6-452e-bf96-f300d94f74ba,19/10/2024 17:21,a_coruna,Chalet adosado en venta en Rúa Paulo Coelho,338.800 €,428.906.606,-84.987.815,193 m²,161 m²,4.0,3.0,sin especificar,En trámite,16.0,https://www.pisos.com/comprar/chalet_adosado-a...,venta
1,ef1933dd-0707-4bd8-bfe8-0d2f00e90cbe,19/10/2024 17:21,a_coruna,Piso en venta en Calle Manuel Beiras,368.000 €,428.697.438,-85.523.869,110 m²,100 m²,3.0,2.0,4ª,En trámite,30.0,https://www.pisos.com/comprar/piso-conxo15706-...,venta
2,a47d9fc7-dd06-4d10-aa01-9022a0636e3a,19/10/2024 17:22,a_coruna,Piso en venta en Calle de la Laguna,165.000 €,43.348.939.902,-8.253.356.896,103 m²,100 m²,3.0,2.0,5ª,En trámite,46.0,https://www.pisos.com/comprar/piso-sada_centro...,venta
3,f6a4795b-36f8-4beb-85c1-3b422df8736d,19/10/2024 17:22,a_coruna,"Casa en venta en Lugar de Alborelle, 15",275.000 €,433.269,-828.635,283 m²,sin especificar,4.0,3.0,sin especificar,En trámite,69.0,https://www.pisos.com/comprar/casa-sada_centro...,venta
4,6fb83620-c719-47c9-a914-1b620206e41d,19/10/2024 17:22,a_coruna,Ático en venta en Calle del Alcalde Domingo Si...,146.700 €,433.155,-838.259,62 m²,60 m²,2.0,2.0,5ª,No indicado,62.0,https://www.pisos.com/comprar/atico-culleredo_...,venta


In [25]:
def limpiar_provincia(df_):
    # Reemplazar guiones bajos por espacios en blanco
    df_['Provincia'] = df_['Provincia'].str.replace('_', ' ')
    
    # Reemplazar 'coruna' por 'coruña'
    df_['Provincia'] = df_['Provincia'].str.replace('coruna', 'coruña', regex=False)
    
    
    return df_

# Aplicar la función al DataFrame
df_ = limpiar_provincia(df_)
df_.head()

,ID,Timestamp,Provincia,Título,Precio,Latitud,Longitud,Superficie Construida,Superficie Útil,Habitaciones,Baños,Planta,Certificado Energético,Número de Fotos,Enlace,venta/alquiler
0,6f85c5bd-5fc6-452e-bf96-f300d94f74ba,19/10/2024 17:21,a coruña,Chalet adosado en venta en Rúa Paulo Coelho,338.800 €,428.906.606,-84.987.815,193 m²,161 m²,4.0,3.0,sin especificar,En trámite,16.0,https://www.pisos.com/comprar/chalet_adosado-a...,venta
1,ef1933dd-0707-4bd8-bfe8-0d2f00e90cbe,19/10/2024 17:21,a coruña,Piso en venta en Calle Manuel Beiras,368.000 €,428.697.438,-85.523.869,110 m²,100 m²,3.0,2.0,4ª,En trámite,30.0,https://www.pisos.com/comprar/piso-conxo15706-...,venta
2,a47d9fc7-dd06-4d10-aa01-9022a0636e3a,19/10/2024 17:22,a coruña,Piso en venta en Calle de la Laguna,165.000 €,43.348.939.902,-8.253.356.896,103 m²,100 m²,3.0,2.0,5ª,En trámite,46.0,https://www.pisos.com/comprar/piso-sada_centro...,venta
3,f6a4795b-36f8-4beb-85c1-3b422df8736d,19/10/2024 17:22,a coruña,"Casa en venta en Lugar de Alborelle, 15",275.000 €,433.269,-828.635,283 m²,sin especificar,4.0,3.0,sin especificar,En trámite,69.0,https://www.pisos.com/comprar/casa-sada_centro...,venta
4,6fb83620-c719-47c9-a914-1b620206e41d,19/10/2024 17:22,a coruña,Ático en venta en Calle del Alcalde Domingo Si...,146.700 €,433.155,-838.259,62 m²,60 m²,2.0,2.0,5ª,No indicado,62.0,https://www.pisos.com/comprar/atico-culleredo_...,venta


In [27]:
# Guardar el DataFrame en un archivo CSV
df_.to_csv('pisos_venta_limpio.csv', index=False)

In [29]:
import pandas as pd

# Cargar los archivos CSV de alquileres y ventas
alquiler = pd.read_csv('pisos_alquiler_limpio.csv', delimiter=',', on_bad_lines='skip', quoting=1)
venta = pd.read_csv('pisos_venta_limpio.csv', delimiter=',', on_bad_lines='skip', quoting=1)

# Agregar la columna 'venta/alquiler' con el valor 'alquiler' para todos los registros de alquileres
alquiler['venta/alquiler'] = 'alquiler'

# Asegurarnos de que ambas tablas tienen las mismas columnas en el mismo orden
# Esto es importante para que la concatenación funcione correctamente
venta = venta[alquiler.columns]

# Fusionar los dos DataFrames
propiedades_final = pd.concat([alquiler, venta], ignore_index=True)

# Guardar el DataFrame combinado en un nuevo archivo CSV
propiedades_final.to_csv('propiedades_final.csv', index=False)


In [1]:
import pandas as pd
from sqlalchemy import create_engine


ruta_archivo = 'C:\\Users\\PC\\Desktop\\Proyecto Final Bootcamp\\propiedades_limpio.csv'

# Ajustar el delimitador según el formato del archivo
df = pd.read_csv(ruta_archivo, delimiter=';', on_bad_lines='skip', quoting=1, encoding='utf-8')

# Mostrar las primeras filas del DataFrame para verificar que las columnas se leen correctamente
print(df.head())

# Crear la conexión con MySQL
usuario = 'root'  
password = 'Leonluca0420!'  
host = 'localhost'  
database = 'pisos_com'  

# Crear el motor de conexión usando SQLAlchemy
engine = create_engine(f'mysql+pymysql://{usuario}:{password}@{host}/{database}')

# Insertar el DataFrame en MySQL
df.to_sql('propiedades_limpio', con=engine, if_exists='replace', index=False)

print("Datos insertados correctamente en la tabla 'propiedades_final'.")


                                     ID         Timestamp Provincia  \
0  2ebc2089-fb32-4dcf-bc1f-48f952f9c7ab  18/10/2024 13:44  a coruña   
1  4677a3e9-e1f0-4d6f-a905-e547184a7b80  18/10/2024 13:44  a coruña   
2  b26707fe-a110-4b78-9bab-ae24afea9a02  18/10/2024 13:44  a coruña   
3  4aa7369a-883f-46f4-9118-da91137cabec  18/10/2024 13:44  a coruña   
4  08526e4e-b6ac-4861-9382-f59e73b23595  18/10/2024 13:44  a coruña   

                                              Título  Precio  \
0                       Piso en alquiler en Ensanche    1600   
1  Piso en alquiler en Avenida de Primo de Rivera, 8    1550   
2  Piso en alquiler en Avenida de Pedro Barrié de...     980   
3  Piso en alquiler en Calle del Comandante Fontanes     950   
4  Apartamento en alquiler en Calle de Nuestra Se...     800   

                 Latitud                Longitud  Superficie Construida  \
0            433.650.629             -84.068.429                  168.0   
1                433.596              